In [12]:
import pandas as pd
import numpy as np

#### Read Data

In [13]:
df = pd.read_csv('D:\FirstSemMT\ML\Assignments\DLBCL.tab',sep='\t')
df.shape

(79, 7071)

In [14]:
df.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,...,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,class
0,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,DLBCL FL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class
2,-1,-45,176,97,-57,233,265,945,56,819,...,1036,-161,1523,-72,897,366,3,76,-28,DLBCL
3,25,-17,531,353,122,155,209,1688,42,639,...,4254,-75,6056,-68,737,81,-4,41,-45,DLBCL
4,73,91,257,80,614,507,760,2252,196,863,...,1934,-129,2901,-9,946,450,38,164,-62,DLBCL


In [15]:
df.drop(df.index[df['class'] == 'DLBCL FL'], inplace = True)
df.drop(df.index[df['class'] == 'class'], inplace = True)
label = df['class'] 
df.drop(['class'], axis = 1, inplace = True)
df = df.values
samples,features = df.shape
print(samples,features)

77 7070


In [16]:
data = df.astype(np.float)
data.shape

(77, 7070)

#### Import train_test_split function and Split dataset into training set and test set

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2,random_state=100) 

#### Find f value for every feature 

In [18]:
total_class = 2

#degree of freedom
dof_between = total_class-1
dof_within = data.shape[0]-total_class
dof_total = dof_between+dof_within

Dof_critical = 3.985   #for alpha=0.05

In [19]:
c1 = np.where(y_train=='DLBCL')[0]
c2 = np.where(y_train=='FL')[0]

#to calculate no. of samples in both the class
c_dash1 = np.where(label=='DLBCL')[0]
c_dash2 = np.where(label=='FL')[0]

#mean of each class
m1 = X_train[c1,:].mean(axis=0)
m2 = X_train[c2,:].mean(axis=0)
m_grand = X_train[:,:].mean(axis=0)

#variance of each class
v1 = X_train[c1,:].var(axis=0)
v2 = X_train[c2,:].var(axis=0)

ss_total = np.sum((X_train[:,:] - m_grand)**2, axis=0)
print(ss_total.shape)

ss_within = (c_dash1.shape[0]-1)*v1 + (c_dash2.shape[0]-1)*v2
ss_between = ((c_dash1.shape[0])*(m1-m_grand)**2) + ((c_dash2.shape[0])*(m2-m_grand)**2)
print(ss_between.shape)

ms_between = ss_between/dof_between
ms_within = ss_within/dof_within

#f-stat
f = ms_between/ms_within
print(f)
print(f[f>Dof_critical].shape)

(7070,)
(7070,)
[4.84007066 1.9016487  0.43630071 ... 0.2652152  0.38814021 0.17751918]
(2495,)


#### Sort indexes in descending order of f value

In [20]:
sort_idxs=(-f).argsort()[:f.shape[0]]
sort_idxs

array([6119,  995, 3968, ..., 1281,  936, 3182], dtype=int64)

#### Select top features (less than 20%)

In [21]:
filt_idxs = sort_idxs[0:int((data.shape[1])*0.2)]
filt_idxs

array([6119,  995, 3968, ..., 4512, 4225, 3194], dtype=int64)

In [22]:
X_new_train = X_train[:,filt_idxs]
X_new_test = X_test[:,filt_idxs]
X_new_train.shape

(61, 1414)

#### KNN Classification

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_new_train, y_train)
y_pred = knn.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using KNN

In [24]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.8125
f1 score: 0.7922077922077921
confusion matrix: [[9 3]
 [0 4]]


#### SNM Classification

In [25]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_new_train, y_train)
y_pred = clf.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using SVM

In [26]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9375
f1 score: 0.9227053140096619
confusion matrix: [[11  1]
 [ 0  4]]
